In [ ]:
import sys
sys.path.append("../")

In [1]:
import numpy as np
import pandas as pd
import torch
import warnings

from model.load import read_csv

from pycox import datasets
from pycox.evaluation import EvalSurv

from model.dataset import Dataset, sample_by_quantiles
from model.fedcox import Federation
from model.net import MLP, MLPPH, CoxPH
from model.discretiser import Discretiser
from model.interpolate import surv_const_pdf_df

from sklearn.model_selection import KFold
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import DataLoader

In [ ]:
rng = np.random.default_rng(12)
_ = torch.manual_seed(12)

In [2]:
df = datasets.rr_nl_nhp.read_df()

Dataset 'rr_nl_nph' not created yet. Making dataset...
Done


In [3]:
df.head()

,x0,x1,x2,duration_true,event_true,censoring_true,duration,event
0,-0.616961,0.244218,-0.124545,14.232236,1.0,0.847494,0.847494,0.0
1,0.570717,0.559952,-0.454815,7.670356,1.0,47.831032,7.670356,1.0
2,-0.447071,0.603744,0.916279,5.810155,1.0,54.283710,5.810155,1.0
3,0.751865,-0.284365,0.001990,5.038248,1.0,7.978237,5.038248,1.0
4,0.366926,0.425404,-0.259498,17.822994,1.0,34.851257,17.822994,1.0


In [ ]:
to_max = ['GRADE','TUMOUR_COUNT','SACT','REGIMEN_COUNT','CLINICAL_TRIAL_INDICATOR','NORMALISED_HEIGHT','NORMALISED_WEIGHT','CHEMO_RADIATION_INDICATOR','DAYS_TO_FIRST_SURGERY','DAYS_SINCE_DIAGNOSIS','BENIGN_BEHAVIOUR']

for m in to_max:
    df = add_aggregate(df,what='max',target=m,groupby='PATIENTID',name=m)
for m in to_median:
    df = add_aggregate(df,what='median',target=m,groupby='PATIENTID',name=m)
for m in to_min:
    df = add_aggregate(df,what='min',target=m,groupby='PATIENTID',name=m)
df.head()